In [1]:
!pip install codecarbon
!pip uninstall -y transformers
!pip install "transformers>=4.46.0"

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)


In [2]:
import json
import os
import re
import time
import random
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, List, Optional
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
    GenerationConfig,
    PreTrainedModel,
)

In [3]:
try:
    from codecarbon import EmissionsTracker
except ImportError:  # pragma: no cover - only hit when codecarbon missing
    EmissionsTracker = None

In [4]:
TEACHER_MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
STUDENT_MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"
DISTILLATION_DATASET_PATH = Path("distillation_dataset.jsonl")
DISTILLED_MODEL_DIR = Path("llama3_2_1b_docstring_distilled")
DEFAULT_NUM_DISTILLATION_SAMPLES = 256
MAX_SOURCE_LENGTH = 2048
MAX_TARGET_LENGTH = 512
MAX_CLEAN_LINES = 60
MIN_DOC_MARKER = ("Args:", "Parameters:", "Returns:")
CODE_BLOCK_PATTERN = re.compile(r"```(?:python)?\s*([\s\S]*?)```", re.IGNORECASE)
DEFAULT_SUMMARY = "Detects if the account balance drops below zero during the provided operations."
CANONICAL_TEMPLATE = """from typing import List

def below_zero(operations: List[int]) -> bool:
    \"\"\"
    {summary}

    Args:
        operations (List[int]): A list of deposit and withdrawal operations.

    Returns:
        bool: True if the balance falls below zero, False otherwise.
    \"\"\"
    balance = 0
    for op in operations:
        balance += op
        if balance < 0:
            return True
    return False
"""
function_code = """
from typing import List

def below_zero(operations: List[int]) -> bool:
    \"\"\"
    You're given a list of deposit and withdrawal operations
    on a bank account that starts with zero balance. Your task is to
    detect if at any point the balance of account falls below zero,
    and at that point function should return True.
    \"\"\"
    balance = 0
    for op in operations:
        balance += op
        if balance < 0:
            return True
    return False
"""


complete_prompt_text = f"""
Please act as an expert Python software engineer. Given the python function below:
{function_code}
I would appreciate it if you could generate a complete and professional Google-style docstring.
The docstring should not include any extra commentary, strictly limited to include the docstring itself and the original function code.
CODE ONLY. Use standard Python indentation. Thank you.
Do not add explanations, notes, or text outside of the code.
Return only the function code with its docstring, without markdown fences or extra text before or after.
"""



In [5]:
def ensure_pad_token(
    tokenizer: PreTrainedTokenizer,
    model: Optional[PreTrainedModel] = None,
    return_model: bool = False,
):
    """
    Ensure tokenizer has a distinct <pad> token.
    If 'model' is provided and we add a new token, resize embeddings and
    mirror pad_token_id into model.config.
    - return_model=False (default): returns tokenizer
    - return_model=True: returns (tokenizer, model)
    """
    added = False
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "<pad>"})
        added = True

    if model is not None:
        if added:
            model.resize_token_embeddings(len(tokenizer))
        model.config.pad_token_id = tokenizer.pad_token_id

    if return_model:
        return tokenizer, model
    return tokenizer
# For TRAINING the student (you MAY add a new <pad> and must resize):
def ensure_pad_for_training(tokenizer, model):
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "<pad>"})
        model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id
    return tokenizer, model

# For INFERENCE with TEACHER (DO NOT change vocab; alias pad→eos):
def ensure_pad_for_inference(tokenizer, model):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    model.config.pad_token_id = tokenizer.pad_token_id
    return tokenizer, model
@dataclass
class PromptResponseRecord:
    prompt: str
    response: str


class PromptResponseDataset(Dataset):
    """
    Converts prompt/response pairs into autoregressive training examples.
    The prompt tokens are masked in the label tensor so that the loss only
    supervises the generated answer.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        records: Iterable[PromptResponseRecord],
        max_source_length: int = MAX_SOURCE_LENGTH,
        max_target_length: int = MAX_TARGET_LENGTH,
    ) -> None:
        self.tokenizer = tokenizer
        self.records = list(records)
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self) -> int:
        return len(self.records)

    def __getitem__(self, index: int) -> Dict[str, List[int]]:
        record = self.records[index]
        source_encoding = self.tokenizer(
            record.prompt,
            truncation=True,
            max_length=self.max_source_length,
            add_special_tokens=False,
        )
        target_encoding = self.tokenizer(
            record.response,
            truncation=True,
            max_length=self.max_target_length,
            add_special_tokens=False,
        )

        input_ids = (
            source_encoding["input_ids"]
            + target_encoding["input_ids"]
            + [self.tokenizer.eos_token_id]
        )

        attention_mask = [1] * len(input_ids)

        labels = (
            [-100] * len(source_encoding["input_ids"])
            + target_encoding["input_ids"]
            + [self.tokenizer.eos_token_id]
        )

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }


def pad_batch(batch: List[Dict[str, List[int]]], pad_token_id: int) -> Dict[str, torch.Tensor]:
    """
    Pads a list of dicts containing tokenized samples into a uniform tensor batch.
    """
    max_length = max(len(item["input_ids"]) for item in batch)
    input_ids: List[List[int]] = []
    attention_masks: List[List[int]] = []
    labels: List[List[int]] = []

    for item in batch:
        pad_length = max_length - len(item["input_ids"])
        input_ids.append(item["input_ids"] + [pad_token_id] * pad_length)
        attention_masks.append(item["attention_mask"] + [0] * pad_length)
        labels.append(item["labels"] + [-100] * pad_length)

    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_masks, dtype=torch.long),
        "labels": torch.tensor(labels, dtype=torch.long),
    }


def load_distillation_records(path: Path) -> List[PromptResponseRecord]:
    if not path.exists():
        return []
    records: List[PromptResponseRecord] = []
    with path.open("r", encoding="utf-8") as fp:
        for line in fp:
            payload = json.loads(line)
            records.append(
                PromptResponseRecord(
                    prompt=payload["prompt"],
                    response=payload["response"],
                )
            )
    return records


def save_distillation_records(path: Path, records: Iterable[PromptResponseRecord]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as fp:
        for record in records:
            json.dump({"prompt": record.prompt, "response": record.response}, fp)
            fp.write("\n")


def _select_candidate_block(blocks: List[str]) -> Optional[str]:
    """Pick the best code block, preferring ones that already contain a docstring."""
    for block in reversed(blocks):
        candidate = block.strip()
        if any(marker in candidate for marker in MIN_DOC_MARKER):
            return candidate
    return blocks[-1].strip() if blocks else None


def _deduplicate_lines(lines: List[str]) -> List[str]:
    deduped: List[str] = []
    for line in lines:
        stripped = line.rstrip()
        if not deduped or stripped != deduped[-1]:
            deduped.append(stripped)
    return deduped


def clean_teacher_response(raw_response: str) -> str:
    """
    Normalizes teacher outputs by keeping only a single cleaned Python code block.
    Falls back to a canonical docstring if the teacher output is malformed.
    """
    raw_response = raw_response.replace("\r\n", "\n")
    matches = CODE_BLOCK_PATTERN.findall(raw_response)
    candidate = _select_candidate_block(matches)

    if not candidate:
        candidate = raw_response.strip()

    lines = candidate.splitlines()
    lines = _deduplicate_lines(lines)[:MAX_CLEAN_LINES]

    cleaned = "\n".join(lines).strip()
    if not cleaned:
        cleaned = CANONICAL_TEMPLATE.format(summary=DEFAULT_SUMMARY)

    return cleaned if cleaned.endswith("\n") else cleaned + "\n"


def collect_teacher_generations(
    prompt_text: str,
    teacher_model_id: str,
    num_samples: int = 4,
    temperature: float = 0.7,
    top_p: float = 0.95,
    max_new_tokens: int = 128,
    device: str = "cuda",
):
    tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)
    model = AutoModelForCausalLM.from_pretrained(
        teacher_model_id,
        torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32),
        low_cpu_mem_usage=True,
    ).to(device).eval()

    print("len(tokenizer), config.vocab_size, emb_size =",
      len(tokenizer),
      getattr(model.config, "vocab_size", None),
      model.get_input_embeddings().weight.shape[0])

    # IMPORTANT: do NOT change teacher vocab size
    tokenizer, model = ensure_pad_for_inference(tokenizer, model)

    # Encode WITHOUT auto BOS/EOS (avoids double-BOS on Llama 3.x)
    encoded = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=False).to(device)

    teacher_gen_cfg = GenerationConfig(
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    outs = []
    with torch.no_grad():
        for _ in range(num_samples):
            gen = model.generate(
                **encoded,
                generation_config=teacher_gen_cfg,
                use_model_defaults=False,  # prevent baked-in overrides (e.g., do_sample/top_p)
            )
            prompt_len = encoded["input_ids"].shape[1]
            new_tokens = gen[0][prompt_len:]
            outs.append(tokenizer.decode(new_tokens, skip_special_tokens=True))
    return outs

def _clean_resp(x: str) -> str:
    try:
        return clean_teacher_response(x)  # noqa: F821
    except NameError:
        return x

def prepare_distillation_dataset(
    prompt_text: str,
    dataset_path: Path = DISTILLATION_DATASET_PATH,
    teacher_model_id: str = TEACHER_MODEL_ID,
    force_regenerate: bool = False,
    num_samples: int = DEFAULT_NUM_DISTILLATION_SAMPLES,
    temperature: float = 0.7,
    top_p: float = 0.95,
    max_new_tokens: int = MAX_TARGET_LENGTH,
    device: str = "cuda",
) -> List[PromptResponseRecord]:
    """
    Loads cached teacher generations if present (unless force_regenerate),
    otherwise collects fresh generations from the teacher, normalizes, and saves JSONL.
    """
    # 1) Load cached if available
    if dataset_path.exists() and not force_regenerate:
        print(f"Loading cached distillation dataset from {dataset_path}")
        records = load_distillation_records(dataset_path)
        cleaned_records = [
            PromptResponseRecord(prompt=r.prompt, response=_clean_resp(r.response))
            for r in records
        ]
        save_distillation_records(dataset_path, cleaned_records)
        return cleaned_records

    # 2) Collect from teacher
    print(
        f"Collecting {num_samples} teacher generations from {teacher_model_id} "
        f"using provided prompt."
    )
    teacher_outputs = collect_teacher_generations(
        prompt_text=prompt_text,
        teacher_model_id=teacher_model_id,
        num_samples=num_samples,
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=max_new_tokens,
        device=device,
    )

    # 3) Normalize to records (PromptResponseRecord list)
    records = [
        PromptResponseRecord(prompt=prompt_text, response=_clean_resp(o))
        for o in teacher_outputs
    ]

    # 4) Save to both the canonical dataset_path and a flat file for inspection
    dataset_path.parent.mkdir(parents=True, exist_ok=True)
    save_distillation_records(dataset_path, records)
    save_distillation_records(Path("distillation_dataset.jsonl"), records)

    print(f"Saved distillation dataset to {dataset_path}")
    return records


# ---------- 1) Train / Distill ----------

def distill_student_from_records(
    records: List[PromptResponseRecord],
    student_model_id: str = STUDENT_MODEL_ID,
    output_dir: Path = DISTILLED_MODEL_DIR,
    num_train_epochs: int = 3,
    per_device_batch_size: int = 1,
    learning_rate: float = 5e-5,
    gradient_accumulation_steps: int = 2,
) -> Path:
    """
    Fine-tunes the student model on the teacher-generated dataset (naive distillation).
    """

    # ---- Load in FP32 for T4 training to avoid AMP/GradScaler issues ----
    tokenizer = AutoTokenizer.from_pretrained(student_model_id, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        student_model_id,
        torch_dtype=torch.float32,      # train in FP32 on T4
        low_cpu_mem_usage=True,
    )

    # Add <pad> if needed and resize embeddings
    tokenizer, model = ensure_pad_for_training(tokenizer, model)  # from earlier message

    # Memory savers
    if hasattr(model, "gradient_checkpointing_enable"):
        model.gradient_checkpointing_enable()  # big saver on T4
    model.config.use_cache = False             # must be False when grad ckpt is on

    # Build dataset AFTER tokenizer finalized
    dataset = PromptResponseDataset(tokenizer, records)

    from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
    output_dir.mkdir(parents=True, exist_ok=True)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8 if torch.cuda.is_available() else None,
    )

    training_args = TrainingArguments(
        output_dir=str(output_dir),
        per_device_train_batch_size=per_device_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        logging_steps=50,
        save_steps=500,
        save_total_limit=2,
        report_to=[],

        # ---- IMPORTANT: no mixed precision (prevents GradScaler.unscale_ path) ----
        fp16=False,
        bf16=False,

        # Optional memory/throughput tuners (safe defaults)
        optim="adamw_torch",
        gradient_checkpointing=True,  # mirrors the manual enable; redundant but explicit
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=data_collator,
    )

    trainer.train()

    # Re-enable cache for inference
    model.config.use_cache = True

    # Save model + tokenizer
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    return output_dir

# ---------- 2) Inference (single, corrected version) ----------

def generate_with_distilled_model(
    prompt_text: str,
    model_path: Path = DISTILLED_MODEL_DIR,
    max_new_tokens: int = MAX_TARGET_LENGTH,
    temperature: float = 0.9,
    top_p: float = 0.95,
    do_sample: bool = True,
    seed: Optional[int] = None,
    model: Optional[torch.nn.Module] = None,
    tokenizer: Optional[PreTrainedTokenizer] = None,
    device: Optional[torch.device] = None,
) -> str:
    if model is None and not model_path.exists():
        raise FileNotFoundError(f"Distilled model directory '{model_path}' not found.")

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    if tokenizer is None:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
    if model is None:
        model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch_dtype, low_cpu_mem_usage=True)

    # Student path: tokenizer may have added <pad>; ensure model matches
    tokenizer, model = ensure_pad_for_training(tokenizer, model)

    model.to(device).eval()
    model.config.use_cache = True

    if seed is not None:
        torch.manual_seed(int(seed))
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(int(seed))

    encoded = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=False).to(device)

    with torch.no_grad():
        out = model(**encoded)
        last_logits = out.logits[:, -1, :]
        if torch.isnan(last_logits).any() or torch.isinf(last_logits).any():
            raise RuntimeError("Invalid logits (NaN/Inf) before generation – tokenizer/model mismatch?")

    gen_cfg = GenerationConfig(
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature if do_sample else None,
        top_p=top_p if do_sample else None,
        num_beams=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    with torch.no_grad():
        gen = model.generate(
            **encoded,
            generation_config=gen_cfg,
            use_model_defaults=False,
        )

    prompt_len = encoded["input_ids"].shape[1]
    new_tokens = gen[0][prompt_len:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True)


def get_model_size_mb(model: torch.nn.Module) -> float:
    torch.save(model.state_dict(), "temp_model.pt")
    size_mb = os.path.getsize("temp_model.pt") / (1024 * 1024)
    os.remove("temp_model.pt")
    return size_mb   # Convert MB to GB for reporting


def run_distilled_inference_trials(
    prompt_text: str,
    model_path: Path = DISTILLED_MODEL_DIR,
    num_runs: int = 5,
    max_new_tokens: int = MAX_TARGET_LENGTH,
    temperature: float = 0.9,
    top_p: float = 0.95,
    base_seed: Optional[int] = None,
    metrics_csv: Path = Path("distilled_inference_metrics.csv"),
    metadata_csv: Path = Path("distilled_model_metadata.csv"),
) -> pd.DataFrame:
    """
    Executes multiple inference runs with the distilled model while monitoring emissions.
    """
    tokenizer = ensure_pad_token(AutoTokenizer.from_pretrained(model_path))
    torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        dtype=torch_dtype,
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    model_size_mb = get_model_size_mb(model)
    if base_seed is None:
        base_seed = (time.time_ns() & 0xFFFFFFFF)
    rows: List[Dict[str, object]] = []
    if EmissionsTracker is None:
        print("CodeCarbon not installed; emissions data will be None.")

    for run_id in range(1, num_runs + 1):
        run_seed = int(base_seed + 9973 * run_id)
        torch.manual_seed(run_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(run_seed)
        tracker = None
        emissions_kg = None

        if EmissionsTracker is not None:
            tracker = EmissionsTracker(save_to_file=False, measure_power_secs=1)
            tracker.start()

        # Logit sanity probe (last-step forward)
        with torch.no_grad():
            encoded = tokenizer(prompt_text, return_tensors="pt", add_special_tokens=False).to(device)
            out = model(**encoded)
            last_logits = out.logits[:, -1, :]
            if torch.isnan(last_logits).any() or torch.isinf(last_logits).any():
                raise RuntimeError("Invalid logits (NaN/Inf) before generation – check tokenizer/model alignment.")

        # Dimensions must align
        vocab = model.get_input_embeddings().weight.shape[0]
        assert vocab == model.config.vocab_size or model.config.vocab_size is None, \
            f"Embedding size {vocab} != config.vocab_size {model.config.vocab_size}"
        assert tokenizer.pad_token_id is not None, "pad_token_id must be set"
        start = time.perf_counter()
        output = generate_with_distilled_model(
            prompt_text=prompt_text,
            model_path=model_path,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,        # ensure sampling
            seed=None,             # seeding handled globally above
            model=model,
            tokenizer=tokenizer,
            device=device,
        )
        inference_time = time.perf_counter() - start

        if tracker is not None:
            emissions_kg = tracker.stop()

        rows.append(
            {
                "run_id": run_id,
                "prompt": prompt_text,
                "output": output,
                "accuracy": "N/A",
                "inference_time_s": inference_time,
                "emissions_kg": emissions_kg,
                "model_size_mb": model_size_mb,
            }
        )

    metrics_df = pd.DataFrame(rows)
    metrics_df.to_csv(metrics_csv, index=False)

    metadata_df = pd.DataFrame(
        [
            {
                "model_path": str(model_path),
                "model_size_mb": model_size_mb,
            }
        ]
    )
    metadata_df.to_csv(metadata_csv, index=False)

    return metrics_df


def run_distillation_workflow(
    prompt_text: str,
    teacher_model_id: str = TEACHER_MODEL_ID,
    student_model_id: str = STUDENT_MODEL_ID,
    dataset_path: Path = DISTILLATION_DATASET_PATH,
    distilled_dir: Path = DISTILLED_MODEL_DIR,
    evaluation_runs: int = 5,
    force_regenerate: bool = False,
    metrics_csv: Path = Path("distilled_inference_metrics.csv"),
    metadata_csv: Path = Path("distilled_model_metadata.csv"),
) -> str:
    """
    End-to-end pipeline that prepares the dataset, fine-tunes the student,
    and returns the distilled model output for the provided prompt.
    """
    records = prepare_distillation_dataset(
        prompt_text=prompt_text,
        dataset_path=dataset_path,
        teacher_model_id=teacher_model_id,
        force_regenerate=force_regenerate,
    )
    print(f"Distillation dataset contains {len(records)} samples.")

    distilled_path = distill_student_from_records(
        records=records,
        student_model_id=student_model_id,
        output_dir=distilled_dir,
    )
    print(f"Saved distilled student model to {distilled_path}")

    metrics_df = run_distilled_inference_trials(
        prompt_text=prompt_text,
        model_path=distilled_path,
        num_runs=evaluation_runs,
        metrics_csv=metrics_csv,
        metadata_csv=metadata_csv,
    )
    if not metrics_df.empty:
        sample = metrics_df.iloc[0]["output"]
        print("=== Distilled Model Output (Run 1) ===")
        print(sample)
        return str(sample)
    return ""

In [6]:
run_distillation_workflow(complete_prompt_text, force_regenerate=False)

Loading cached distillation dataset from distillation_dataset.jsonl
Distillation dataset contains 256 samples.


`torch_dtype` is deprecated! Use `dtype` instead!
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Step,Training Loss
50,0.224500
100,0.132800
150,0.093000
200,0.075800
250,0.077400
300,0.055900
350,0.053000


Saved distilled student model to llama3_2_1b_docstring_distilled


[codecarbon WARNING @ 00:29:32] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 00:29:32] [setup] RAM Tracking...
[codecarbon INFO @ 00:29:32] [setup] CPU Tracking...
[codecarbon WARNING @ 00:29:33] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 00:29:33] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 00:29:33] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 00:29:33] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 00:29:33] [setup] GPU Tracking...
[codecarbon INFO @ 00:29:33] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 00:29:33] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

=== Distilled Model Output (Run 1) ===
from typing import List

def below_zero(operations: List[int]) -> bool:
    """
    You're given a list of deposit and withdrawal operations
    on a bank account that starts with zero balance. Your task is to
    detect if at any point the balance of account falls below zero,
    and at that point function should return True.

    Args:
        operations (List[int]): A list of deposit and withdrawal operations.
            Positive numbers represent deposits, while negative numbers represent withdrawals.

    Returns:
        bool: True if the balance falls below zero at any point, False otherwise.

    Examples:
        >>> below_zero([1, -2, 3, -4])
        False
        >>> below_zero([-1, 2, -3])
        True
    """
    balance = 0
    for op in operations:
        balance += op
        if balance < 0:
            return True
    return False



'from typing import List\n\ndef below_zero(operations: List[int]) -> bool:\n    """\n    You\'re given a list of deposit and withdrawal operations\n    on a bank account that starts with zero balance. Your task is to\n    detect if at any point the balance of account falls below zero,\n    and at that point function should return True.\n\n    Args:\n        operations (List[int]): A list of deposit and withdrawal operations.\n            Positive numbers represent deposits, while negative numbers represent withdrawals.\n\n    Returns:\n        bool: True if the balance falls below zero at any point, False otherwise.\n\n    Examples:\n        >>> below_zero([1, -2, 3, -4])\n        False\n        >>> below_zero([-1, 2, -3])\n        True\n    """\n    balance = 0\n    for op in operations:\n        balance += op\n        if balance < 0:\n            return True\n    return False\n'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

metrics_csv = Path("distilled_inference_metrics.csv")
metadata_csv = Path("distilled_model_metadata.csv")

!cp "{metrics_csv}" "/content/drive/My Drive/"
!cp "{metadata_csv}" "/content/drive/My Drive/"

print(f"Downloaded {metrics_csv} and {metadata_csv} to Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Downloaded distilled_inference_metrics.csv and distilled_model_metadata.csv to Google Drive.
